In [ ]:
%matplotlib widget
import copy
import datetime
import json
import os
import scipy.spatial
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload

import celeri
celeri = reload(celeri)

In [ ]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
with open('./data/western_north_america/command.json', 'r') as f:
    command = json.load(f)
station = pd.read_csv(command["station_file_name"])
segment = pd.read_csv(command["segment_file_name"])
block = pd.read_csv(command["block_file_name"])

In [ ]:
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command)

In [ ]:
np_segments = np.zeros((len(segment), 2, 2))
np_segments[:, 0, 0] = segment.lon1.to_numpy()
np_segments[:, 1, 0] = segment.lon2.to_numpy()
np_segments[:, 0, 1] = segment.lat1.to_numpy()
np_segments[:, 1, 1] = segment.lat2.to_numpy()

In [ ]:
# De-duplicate the vertices and build an ptr_edge array
all_vertices = np_segments.reshape((-1, 2))
tree = scipy.spatial.KDTree(all_vertices, leafsize=1000)
duplicates = tree.query_ball_point(all_vertices, 1e-8)

ptr_edges = []
dedup_vertices = []
original_to_new = dict()
for i in range(np_segments.shape[0]):
    v1_idx = duplicates[2 * i][0]
    v2_idx = duplicates[2 * i + 1][0]
    if v1_idx == 2 * i:
        original_to_new[2 * i] = len(dedup_vertices)
        dedup_vertices.append(np_segments[i][0])
    if v2_idx == 2 * i + 1:
        original_to_new[2 * i + 1] = len(dedup_vertices)
        dedup_vertices.append(np_segments[i][1])
    ptr_edges.append((original_to_new[v1_idx], original_to_new[v2_idx]))

np_dedup_vertices = np.array(dedup_vertices)

# Check that the vertices are unique up to 1e-8 now.
new_tree = scipy.spatial.KDTree(np_dedup_vertices)
np.all([v[0] == i for i,v in enumerate(new_tree.query_ball_point(np_dedup_vertices, 1e-8))])

# Build the graph
G = nx.Graph()
for e in ptr_edges:
    G.add_edge(*e)

# Check that it is planar and build the PlanarEmbedding data structure.
is_planar, planar_embedding = nx.check_planarity(G)

# Extract each polygon. 
counted_half_edges = set()
polygons = []
for component in nx.connected_components(planar_embedding):
    for v in component:
        for w in planar_embedding.neighbors_cw_order(v):
            if (v, w) not in counted_half_edges:
                polygons.append(planar_embedding.traverse_face(v, w, counted_half_edges))

print("Found " + str(len(polygons)) + " closed polygons")

# Map polygons boundaries back to 2 labesl for each block
n_vertices = np_dedup_vertices.shape[0]
n_edges = len(ptr_edges)

vertex_to_polygon = [[] for i in range(n_vertices)] 
for p_idx, p in enumerate(polygons):
    for v_idx in p:
        vertex_to_polygon[v_idx].append(p_idx)

edge_to_polygon = []
for e in ptr_edges:
    polys_v0 = vertex_to_polygon[e[0]]
    polys_v1 = vertex_to_polygon[e[1]]
    polys_edge = [p_idx for p_idx in polys_v0 if p_idx in polys_v1] # Intersection of lists
    edge_to_polygon.append(polys_edge)

segment["initial_label1"] = np.array(edge_to_polygon)[:, 0]
segment["initial_label2"] = np.array(edge_to_polygon)[:, 1]

In [ ]:
longitude_east_offset = 1e-3
unprocessed_value = 99
correct_east_labels = unprocessed_value * np.ones(len(segment))
correct_west_labels = unprocessed_value * np.ones(len(segment))

for current_block_label in range(len(polygons)):
    current_block_idx = np.union1d(segment.index[segment.initial_label1 == current_block_label],
                                   segment.index[segment.initial_label2 == current_block_label])
    segment_current_block = segment.iloc[current_block_idx]
    polygon_vertex_idx = polygons[current_block_label]
    vertices_current_block = np.concatenate([np_dedup_vertices[polygon_vertex_idx], np_dedup_vertices[polygon_vertex_idx[0]][None,:]])
    is_midpoint_inpolygon = celeri.inpolygon(segment_current_block.mid_lon_plate_carree.values + longitude_east_offset,
                                segment_current_block.mid_lat_plate_carree.values,
                                vertices_current_block[:, 0], vertices_current_block[:, 1])
    correct_east_labels[current_block_idx[np.where(is_midpoint_inpolygon == True)]] = current_block_label
    correct_west_labels[current_block_idx[np.where(is_midpoint_inpolygon == False)]] = current_block_label

# Find unprocessed indices
unprocessed_idx = np.union1d(np.where(correct_east_labels == unprocessed_value),
                             np.where(correct_west_labels == unprocessed_value))

# plt.figure()
# plt.title("Segment indices")
# for i in range(len(segment)):
#     plt.plot([segment.lon1.values[i], segment.lon2.values[i]],
#              [segment.lat1.values[i], segment.lat2.values[i]], "-k")
#     plt.text(segment.mid_lon_plate_carree.values[i] - 0.25,
#              segment.mid_lat_plate_carree.values[i],
#              str(i), fontsize=6)

plt.figure()
plt.title("Initial labels")
for i in range(len(segment)):
    plt.plot([segment.lon1.values[i], segment.lon2.values[i]],
             [segment.lat1.values[i], segment.lat2.values[i]], "-k")
    plt.text(segment.mid_lon_plate_carree.values[i] - 0.25,
             segment.mid_lat_plate_carree.values[i],
             str(segment.initial_label1[i]), fontsize=16)
    plt.text(segment.mid_lon_plate_carree.values[i] + 0.25,
             segment.mid_lat_plate_carree.values[i],
             str(segment.initial_label2[i]), fontsize=16)

plt.figure()
plt.title("Attempt at east and west labels")
for i in range(len(segment)):
    plt.plot([segment.lon1.values[i], segment.lon2.values[i]],
             [segment.lat1.values[i], segment.lat2.values[i]], "-k")
    plt.text(segment.mid_lon_plate_carree.values[i] - 0.25,
             segment.mid_lat_plate_carree.values[i],
             str(correct_west_labels[i]), fontsize=6)
    plt.text(segment.mid_lon_plate_carree.values[i] + 0.25,
             segment.mid_lat_plate_carree.values[i],
             str(correct_east_labels[i]), fontsize=6)

for i in range(len(nan_idx)):
    plt.plot([segment.lon1.values[nan_idx[i]], segment.lon2.values[nan_idx[i]]],
             [segment.lat1.values[nan_idx[i]], segment.lat2.values[nan_idx[i]]], "-r")


In [ ]:
correct_east_labels[1] == 99